<h1> Summary </h1>
This is a simple demo of a basic DNN model to classify a customer as going to Churn or not. Churn means to leave our bussiness as a customer whether or not they return later. This is an example of a simple logistic regression DNN with 3 layers (RELU->RELU->Sigmoid). I have heavily commented this notebook to explain almost every line of code as it's written. With the following code I was able to reach an accuracy of 78.6%

<h1>0. Import Data</h1>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

*   read in the Churn.csv data
*   download the csv from https://github.com/nicknochnack/Tensorflow-in-10-Minutes/blob/main/Churn.csv
*   copy or save it to the root folder of this project
*   you can also use your own data as long as it has at least 1 feature column and a label column named Churn

In [ ]:
df = pd.read_csv('Churn.csv')
df.head()


*   Drop the CustomerID and Churn columns from the features dataframe.
*   The Churn column becomes the label.
*   Use a lambda to map the string "Yes" to 1 and other values ie("No") to 0

In [7]:
x = pd.get_dummies(df.drop(['Churn','Customer ID'],axis=1))
y = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)


*   Split the data into training and test sets

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [ ]:
x_train.head()

In [ ]:
y_train.head()

<h1>1. Import Dependancies</h1>

*   Sequential is the basic model that has 1 or more layers
*   load model lets us load a saved model
*   Dense layers are fully connected layers inside the network we build
*   Accuracy is a metric that is (TP+TN)/(TP+TN+FN+FP)

In [11]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import  Dense
from sklearn.metrics import accuracy_score

<h1>2. Build and Compile Model</h1>

*   create the model
*   add the first and second layers with a relu activation function
  *   relu is essentially y = max(0,x) so anything before x = 0 becomes 0 <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/ReLU_and_GELU.svg/1200px-ReLU_and_GELU.svg.png"> image from https://en.wikipedia.org/wiki/Rectifier_%28neural_networks%29
*   add the final layer with a sigmoid activation function
  *   sigmoid takes continous values and puts them into the range of 0 -> 1 making them probablities
  <img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/88/Logistic-curve.svg/1200px-Logistic-curve.svg.png"> image from https://en.wikipedia.org/wiki/Sigmoid_function

In [12]:
model = Sequential()
model.add(Dense(units=32, activation='relu', input_dim=len(x_train.columns)))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1,activation="sigmoid"))

*   Compile the model using binary cross_etropy ass a loss function
  *   binary cross entropy: -1/n*Sum{1->n}(Yi(log(Y'i))+(1-Yi)(log(1-Y'i))
*   use accuracy metrics to evaluate how well model performs

In [13]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics='accuracy')

<h1>3. Fit Predict and Evaluate</h1>

*   we pass in our training features x and our labels y
*   we then set hyperparameters Epochs and batch size
  *  epochs are how long we train for(one forward pass and one backward pass of the training examples)
  *  batch size how many examples to use at once

In [ ]:
EPOCHS = 200
BATCH_SIZE = 32
model.fit(x_train,y_train,epochs=EPOCHS, batch_size=BATCH_SIZE)

*   predict over our test set and set the value to 0 or 1 based on if it is over or bellow our threshold value

In [15]:
Threshold = 0.5
test_predictions = model.predict(x_test)
test_predictions = [0 if value < Threshold else 1 for value in test_predictions]

45/45 [==============================] - 0s 1ms/step


*   calcuate the accuracy score of the predictions

In [16]:
accuracy_score(y_test,test_predictions)

0.7863733144073811

<h1>4. Saving and Reloading</h1>

*   save model to folder named churn_model

In [ ]:
model.save('churn_model')

*   delete and reload model from disk

In [19]:
del model
model = load_model('churn_model')